<a href="https://colab.research.google.com/github/g-aditi/moral-compass/blob/Nihaarika/Moral_Compass_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [3]:
import PyPDF2
import pandas as pd
import numpy as np
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
def pdf_to_text(pdf_data_directory, txt_data_directory):
    # Converting pdf version paper to txt file

    if not os.path.exists(txt_data_directory):
        os.makedirs(txt_data_directory)

    for pdf_filename in os.listdir(pdf_data_directory):

        if pdf_filename.endswith(".pdf"):

            pdf_path = os.path.join(pdf_data_directory, pdf_filename)
            #checkpoint
            print (pdf_filename)
            txt_filename = os.path.splitext(pdf_filename)[0] + ".txt"
            txt_path = os.path.join(txt_data_directory, txt_filename)

            with open(pdf_path, "rb") as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text += page.extract_text()

                with open(txt_path, "w", encoding="utf-8") as txt_file:
                    txt_file.write(text)

pdf_data_directory = "/content/sample_data/MoralCompass_Papers_VDB_pdf"
txt_data_directory = "/content/sample_data/MoralCompass_Papers_VDB_txt"
pdf_to_text(pdf_data_directory, txt_data_directory)



the-belmont-report-508c_FINAL.pdf
Subpart E — Registration of Institutional Review Boards _ HHS.gov.pdf
Subpart C — Additional Protections Pertaining to Biomedical and B _ HHS.gov.pdf
Annotated Comparison of the Pre-2018 Common Rule with _ HHS.gov.pdf
Terminology _ HHS.gov.pdf
Human Subject Regulations Decision Charts_ 2018 Requirements _ HHS.gov.pdf
November 21, 2019_ Determination of Exception for Certain HHS _ HHS.gov.pdf
Pre-2018 Requirements _ HHS.gov.pdf
Single IRB Exception Determinations _ HHS.gov.pdf
eCFR __ 45 CFR Part 46 (July 19, 2018) -- Protection of Human Subjects.pdf
466535.pdf
Revised Common Rule _ HHS.gov.pdf
32843821.pdf
Subpart D — Additional Protections for Children Involved _ HHS.gov.pdf
Excerpts from the January 19, 2017 Revised Common Rule Preamble _ HHS.gov.pdf
Procedures-for-HSR-2022.pdf
Human Subject Regulations Decision Charts_ Pre-2018 Requirements _ HHS.gov.pdf
Subpart B - Additional Protections for Pregnant Women, Human _ HHS.gov.pdf
OHRP Expedited Review

In [18]:
# training a document vectorizer model

txt_data_directory = "/content/sample_data/MoralCompass_Papers_VDB_txt"
txt_file_paths = [os.path.join(txt_data_directory, file) for file in os.listdir(txt_data_directory) if file.endswith('.txt')]
tagged_data = []
for i, txt_file_path in enumerate(txt_file_paths):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
         text = file.read()
         words = word_tokenize(text)
         words = [word.lower() for word in words]
         tagged_data.append(TaggedDocument(words, tags=['doc_' + str(i)]))
         model = Doc2Vec(vector_size=20, epochs=300)
         model.build_vocab(tagged_data)
         model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

document_vectors = [model.dv['doc_' + str(i)] for i in range(len(tagged_data))]
#checkpoint
print (document_vectors)

[array([-1.8662444 ,  0.5307063 ,  0.2298599 ,  1.8381761 ,  0.9464375 ,
        1.4622895 , -0.55958086, -0.8884981 , -4.260886  , -1.7389991 ,
        0.6425447 ,  1.3934507 , -1.3328843 ,  0.7960754 , -2.1334176 ,
        0.7252287 ,  4.424612  , -1.2089963 , -2.0054047 , -1.0875001 ],
      dtype=float32), array([-3.0624354 , -6.959306  , -0.8113659 ,  1.7420998 , -3.019377  ,
        0.1592336 , -3.9212346 , -0.6054859 ,  2.485858  , -1.7885183 ,
        2.091048  ,  3.3359594 , -4.250892  , -0.35127908, -5.400684  ,
        0.44479668,  2.146578  ,  3.9900775 , -4.816547  ,  2.6008701 ],
      dtype=float32), array([ -1.3399322 , -11.827755  ,  -0.17848131,   0.97681904,
        -4.2431917 ,   1.0926656 ,  -1.7549692 ,   2.3094811 ,
         2.0618014 ,  -0.75689465,   1.0766501 ,   7.9966316 ,
        -5.6240134 ,   2.3998873 ,  -9.4388075 ,   0.53911144,
         2.750464  ,   2.7980878 ,  -2.8580332 ,   4.594814  ],
      dtype=float32), array([ 4.4670148e+00,  7.1317816e+00, 

In [19]:
!pip install pandas sentence-transformers pandarallel faiss-cpu



In [20]:
import faiss

In [22]:
vector_dim = len(document_vectors[0])
#checkpoint
print (vector_dim)
index = faiss.IndexFlatL2(vector_dim)
vectors_np = np.array(document_vectors).astype('float32')

20
